In [1]:
import numpy as np
import pandas as pd
import math

low_memory = False

# Read csv into pandas dataframe

df = pd.read_csv('train.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
np.sort(df['VisitorTeamAbbr'].unique())

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [3]:
np.sort(df['HomeTeamAbbr'].unique())

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [4]:
np.sort(df['PossessionTeam'].unique())

array(['ARZ', 'ATL', 'BLT', 'BUF', 'CAR', 'CHI', 'CIN', 'CLV', 'DAL',
       'DEN', 'DET', 'GB', 'HST', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [5]:
df['FieldPosition'].unique()

array(['NE', 'KC', nan, 'BUF', 'NYJ', 'ATL', 'CHI', 'CIN', 'BLT', 'CLV',
       'PIT', 'ARZ', 'DET', 'JAX', 'HST', 'TEN', 'OAK', 'WAS', 'PHI',
       'LA', 'IND', 'GB', 'SEA', 'CAR', 'SF', 'DAL', 'NYG', 'NO', 'MIN',
       'DEN', 'LAC', 'TB', 'MIA'], dtype=object)

In [6]:
df = df.replace({'PossessionTeam': {'BLT': 'BAL', 'CLV': 'CLE', 'HST': 'HOU', 'ARZ': 'ARI'}})
df = df.replace({'FieldPosition': {'BLT': 'BAL', 'CLV': 'CLE', 'HST': 'HOU', 'ARZ': 'ARI'}})

In [7]:
pd.set_option('display.max_columns', None)

df['FieldPosition'].unique()

array(['NE', 'KC', nan, 'BUF', 'NYJ', 'ATL', 'CHI', 'CIN', 'BAL', 'CLE',
       'PIT', 'ARI', 'DET', 'JAX', 'HOU', 'TEN', 'OAK', 'WAS', 'PHI',
       'LA', 'IND', 'GB', 'SEA', 'CAR', 'SF', 'DAL', 'NYG', 'NO', 'MIN',
       'DEN', 'LAC', 'TB', 'MIA'], dtype=object)

In [8]:
# Kaggle data modified with adjusted Player Names
kaggle_df = df.copy()
kaggle_df['DisplayName'] = kaggle_df['DisplayName'].str.lower().str.replace(" ", ""
                                                                ).str.replace('[^\w\s]',''
                                                                             )
# Madden Data loaded
nfl_17 = pd.read_csv("nfl_rating_17 v2.csv")
nfl_18 = pd.read_csv("nfl_rating_18 v2.csv")
nfl_19 = pd.read_csv("nfl_rating_19 v2.csv")

In [9]:
# Player name isolation based on season
kaggle_17 = kaggle_df[kaggle_df['Season']==2017]
name_17 = kaggle_17['DisplayName'].drop_duplicates().to_list()

kaggle_18 = kaggle_df[kaggle_df['Season']==2018]
name_18 = kaggle_18['DisplayName'].drop_duplicates().to_list()

kaggle_19 = kaggle_df[kaggle_df['Season']==2019]
name_19 = kaggle_19['DisplayName'].drop_duplicates().to_list()

In [10]:
# Players that will obtain Madden data
yes_show_17 = []
for i in name_17:
    if i in nfl_17['name'].tolist():
        yes_show_17.append(i)

yes_show_18 = []
for i in name_18:
    if i in nfl_18['name'].tolist():
        yes_show_18.append(i)

yes_show_19 = []
for i in name_19:
    if i in nfl_19['name'].tolist():
        yes_show_19.append(i)

In [11]:
# Madden columns currently being used for merge

column_order = ['name','awareness', 'strength', 'ball carrier vision', 'acceleration', 
                'run block', 'tackle', 'truck', 'speed', 'impact blocking', 'block shedding']

# column_order = ['name','awareness',
#        'strength', 'ball carrier vision',
#        'pursuit', 'acceleration', 'spin move',
#        'finesse moves', 'run block', 'tackle',
#        'zone coverage', 'weight', 'truck', 'juke move',
#        'play recognition', 'speed', 'run block power', 'jumping', 'toughness',
#        'man coverage', 'stiff arm', 'power moves',
#        'release', 'hit power',
#        'pass block power', 'impact blocking', 'stamina', 'carrying','pass block', 'press',
#        'block shedding', 'run block finesse', 'agility', 'elusiveness',
#        'pass block finesse', 'route running']

In [12]:
# Madden datasets adjusted to show names that appear in Kaggle dataset
# Kaggle dataset has 'DisplayName' column renamed to 'name' for merger purposes

nfl_17_mod = nfl_17[nfl_17['name'].isin(yes_show_17)].drop(columns=['Unnamed: 0'])
kaggle_17 = kaggle_17.rename(columns={'DisplayName':'name'})
nfl_17_mod = nfl_17_mod[column_order]

nfl_18_mod = nfl_18[nfl_18['name'].isin(yes_show_18)].drop(columns=['Unnamed: 0'])
kaggle_18 = kaggle_18.rename(columns={'DisplayName':'name'})
nfl_18_mod = nfl_18_mod[column_order]

nfl_19_mod = nfl_19[nfl_19['name'].isin(yes_show_19)].drop(columns=['Unnamed: 0'])
kaggle_19 = kaggle_19.rename(columns={'DisplayName':'name'})
nfl_19_mod = nfl_19_mod[column_order]

In [13]:
# Merge between Madden and Kaggle 
nfl_17_res = kaggle_17.merge(nfl_17_mod, on='name', how='left')
nfl_18_res = kaggle_18.merge(nfl_18_mod, on='name', how='left')
nfl_19_res = kaggle_19.merge(nfl_19_mod, on='name', how='left')

kaggle_modified = pd.concat([nfl_17_res, nfl_18_res, nfl_19_res], ignore_index=True)

In [14]:
kaggle_modified

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,name,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,awareness,strength,ball carrier vision,acceleration,run block,tackle,truck,speed,impact blocking,block shedding
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,ericberry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,0.93,0.66,0.81,0.90,0.45,0.79,0.55,0.91,0.68,0.65
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,allenbailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,0.77,0.88,0.40,0.84,0.45,0.79,0.54,0.80,0.86,0.79
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,justinhouston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,0.90,0.85,0.10,0.87,0.45,0.92,0.40,0.84,0.86,0.83
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,derrickjohnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,0.90,0.73,0.35,0.86,0.45,0.88,0.25,0.82,0.88,0.71
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,ronparker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,0.86,0.67,0.64,0.90,0.45,0.69,0.20,0.90,0.64,0.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689967,2019112500,20191125003789,home,69.58,31.33,4.19,1.33,0.43,92.32,20.94,2556593,corylittleton,58,2019,38,4,04:11:00,BAL,2,9,BAL,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,6-3,228,11/18/1993,Washington,ILB,LA,BAL,12,Los Angeles Memorial Coliseum,"Los Angeles, CA",Outdoor,Grass,Clear,62.0,64.0,5,WSW,0.85,0.73,0.22,0.87,0.45,0.83,0.38,0.83,0.69,0.70
689968,2019112500,20191125003789,home,72.16,25.09,1.01,1.40,0.08,65.83,76.48,2558199,tanzelsmart,92,2019,38,4,04:11:00,BAL,2,9,BAL,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,6-1,295,11/06/1994,Tulane,DT,LA,BAL,12,Los Angeles Memorial Coliseum,"Los Angeles, CA",Outdoor,Grass,Clear,62.0,64.0,5,WSW,0.68,0.82,0.24,0.79,0.45,0.74,0.35,0.67,0.81,0.62
689969,2019112500,20191125003789,home,72.23,31.95,1.75,0.68,0.17,93.99,90.26,2558830,samsonebukam,50,2019,38,4,04:11:00,BAL,2,9,BAL,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03

In [15]:
# data = pd.read_csv("Kaggle_Madden.csv")
# data = data.drop(columns=['Unnamed: 0'])
# pd.set_option('display.max_columns', None)

### Adjusted Dataset Cleaning

In [16]:
games = kaggle_modified['PlayId'].drop_duplicates().to_list()
sim_df = kaggle_modified[['PlayId','name', 'Position']]
new_data = []
og_data = kaggle_modified.copy()
for i in games:
    curr = sim_df.loc[sim_df['PlayId']==i]['name'].value_counts()
    if curr.sum() > 22: # over 11 players per side
        curr_data = og_data.loc[og_data['PlayId'] == i]
        curr_data = curr_data.drop_duplicates(subset=['name', 'NflId'], keep='last') # drops duplicate names, maintains unique ID
        new_data.append(curr_data)
    elif curr.sum() < 23: # standard # of players added to list
        curr_data = og_data.loc[og_data['PlayId'] == i]
        new_data.append(curr_data)

# new_data contains individual dataframes 
new_df = pd.concat(new_data)
new_df = new_df.reset_index(drop=True)

In [17]:
len(new_data)

31007

In [18]:
# original kaggle dataset rows = 682154
new_df.shape

(682154, 59)

#### Dataset Check

In [19]:
nfl_kaggle = pd.read_csv("train.csv", low_memory=False)

In [20]:
games = kaggle_modified['PlayId'].drop_duplicates().to_list()
missing_data = {}
for i in games:
    new_len = new_df.loc[new_df['PlayId']==i]['name'].value_counts().sum()
    old_len = nfl_kaggle.loc[nfl_kaggle['PlayId']==i]['DisplayName'].value_counts().sum()
    missing_data[i] = new_len+old_len

In [21]:
# should return nothing if everything is going well
for i in missing_data:
    if missing_data[i] != 44:
        print(i)
        print(missing_data[i])

In [22]:
new_df.to_csv("Kaggle_Madden.csv")

In [23]:
# Resulting merge dataframe to CSV file
# nfl_17_res.to_csv("kaggle_17_Complete.csv")
# nfl_18_res.to_csv("kaggle_18_Complete.csv")
# nfl_19_res.to_csv("kaggle_19_Complete.csv")